<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT23/blob/main/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Approximation**
**Teo Nordström**

# **Abstract**

This file contains the solutions to the mandatory problem from Lab3 in DD2363, in addition to the solution to one of the optional problems. It is based upon pseudocode and info found in *Methods in Computational Science* by Johan Hoffman (2021)

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [ ]:
"""This file is based on a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# TEMPLATE INFO:
# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# CODE INFO:
# Code written by Teo Nordström 2024, no license.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

These are the neccessary modules for everything in this file to work.

In [ ]:
from google.colab import files

import numpy as np



Also set up some functions from Lab 1 that will be used

In [ ]:
def modified_gram_schmidt_iteration(A:np.ndarray):
    n = len(A)
    Q = np.zeros((n, n))
    R = np.zeros((n, n))
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], v)
            v = v - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]
    return Q, R


def backward_substitution(U:np.ndarray, b:np.ndarray):
    n = U.shape[1]
    x = np.zeros(n)
    x[n-1] = b[n-1] / U[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum += U[i, j] * x[j]
        x[i] = (b[i] - sum) / U[i, i]
    return x


def direct_solver(A:np.ndarray, b:np.ndarray):
    Q, R = modified_gram_schmidt_iteration(A)
    y = np.transpose(Q).dot(b)
    x = backward_substitution(R, y)
    return x

# **Introduction**

All solutions will be partially or entirely based upon the book *Methods in Computational Science* by Johan Hoffman (2021). In the text, it will be referred to as the "course book".

# L2 Projection over Mesh
The L2 Projection over Mesh is a way to get a piecewise linear interpolation over a mesh of points. This is done using basis functions based upon local Lagrange shape functions. This can create approximations of not only continuous but also discrete functions which is why it is desirable.



# **Method**

# L2 Projection over Mesh
To produce the L2 Projection we use Algorithm 9.2 that is defined in the course book. It assembles the matrix $A$ and vector $b$ so we can solve for $x$ to get the projection. We go through each point $k$ in the input mesh to get a fit for the function. We know that since this is a 1D mesh each point only has two shape functions meaning $q = 2$. For each $k$ we create a local vector and matrix. These are calculated using integration, which in this case is quite simple considering that there is only two dimensions. While the values in $b^k$ have to be based upon the function through $b^k_i = \int_{I_k} f(x)\lambda_{k,i}(x)dx$ where $\lambda_{k,i}(x)$ are the local shape functions (which in 2D can be represented simply as the function shows), the integrate matrix only has to worry about the interval length. From chapter 9.5 in the course book we can learn that the tridiagonal matrix has a value of $\frac{2h}{3}$ in the diagonal and $\frac{h}{6}$ on the off-diagonals. Since we add the local matrices together on the global one this will automatically happen if we set the local matrices to have $\frac{h}{3}$ in the diagonal and $\frac{h}{6}$ on the off-diagonals.

After all this is done, we use the solver that was created in Lab 1 to solve the $Ax = b$ equation to get the interpolated function values.

In [ ]:
def integrate_vector(f, k, i, p):
    if i == 0:
        return f(p[k]) * (p[k+1] - p[k]) / 2
    return f(p[k + 1]) * (p[k+1] - p[k]) / 2


def integrate_matrix(k, i, j, p):
    if i == j:
        return (p[k+1] - p[k]) / 3
    return (p[k+1] - p[k]) / 6


def add_vector_at_pos(b, b_k, k):
    b[k] += b_k[0]
    b[k + 1] += b_k[1]
    return b


def add_matrix_at_pos(A, a_k, k):
    A[k, k] += a_k[0, 0]
    A[k+1, k] += a_k[1, 0]
    A[k, k+1] += a_k[0, 1]
    A[k+1, k+1] += a_k[1, 1]
    return A


def assemble_system(f, points):
    n = len(points)
    A = np.zeros((n, n))
    b = np.zeros(n)
    for k in range(n - 1):
        q = 2
        b_k = np.zeros(q)
        a_k = np.zeros((q, q))
        for i in range(q):
            b_k[i] = integrate_vector(f, k, i, points)
            for j in range(q):
                a_k[i, j] = integrate_matrix(k, i, j, points)
        b = add_vector_at_pos(b, b_k, k)
        A = add_matrix_at_pos(A, a_k, k)
    return A, b


def L2_projection(f, points):
    A, b = assemble_system(f, points)
    return direct_solver(A, b)


def as_f(x):
    return x ** 2


print(L2_projection(as_f, [0, 1, 2, 3, 4]))

[-0.28571429  0.57142857  4.          7.42857143 20.28571429]


# **Results**

In this section tests will be performed to verify that the solutions are correct

# L2 Projection over Mesh
To test our L2 Projection over Mesh function we build up an equation to check likeness. This will be either a continuous second degre equation or a discrete linear equation. We then take a number of equally spaced points $(-15 ≤ p ≤ 15)$ to create the projection. We then create a hundred random new points within this range to test how close the value actually is. We use linear interpolation to find the exact value in the projection and add the difference to a sum. We divide this sum by the amount of points to get the total difference. We can see that this is relatively low, especially when increasing amount of points, meaning the projection follows the function quite closely.

To see convergence against the exact function we can look at how the accuracy gets better when more points are added, which we easily can see is occurring. A test with $10000$ points ($n=10000$) was also performed, which took forever to process and led to a result that was a value within a rounding error.

In [ ]:
def L2_test(iters, discrete=False):
    n = 64

    if discrete:
        def f(x):
            if int(x) % 2 == 0:
                return 0
            return 10
    else:
        a = (np.random.randint(1, 5))
        b = (np.random.randint(1, 5))

        def f(x):
            return a * x ** 2 + b * x

    for test in range(iters):

        points = [i*(30/n) - 15 for i in range(n)]
        L2 = L2_projection(f, points)
        sumdiff = 0
        for i in range(100):
            x = np.random.rand() * 30 - 15
            ind = int((n * (x + 15)) / 30)
            if ind == n - 1:
                ind -= 1
            x1 = points[ind]
            x2 = points[ind + 1]
            y1 = L2[ind]
            y2 = L2[ind + 1]
            val_L2 = y1 * ((x2 - x) / (x2 - x1)) + y2 * ((x - x1) / (x2 - x1))
            sumdiff += abs(f(x) - val_L2)
        print(f"Test {test+1}, {'Discrete, ' if discrete else ''}{n} points: Avg error in 100 points = {sumdiff / 100}")
        n *= 2


L2_test(5)

Test 1, 64 points: Avg error in 100 points = 1.2299472717414128
Test 2, 128 points: Avg error in 100 points = 0.07508151560824813
Test 3, 256 points: Avg error in 100 points = 0.0037686550406729833
Test 4, 512 points: Avg error in 100 points = 0.026498554935222705
Test 5, 1024 points: Avg error in 100 points = 0.00014177784402226058


One of the things L2 Projection can do well is also process discrete functions. We can see that while they have a greater error, they also converge towards a low value when the amount of points are increased. This wouldn't be possible with simpler interpolation schemes.

In [ ]:
L2_test(5, True)

Test 1, Discrete, 64 points: Avg error in 100 points = 2.9107675605908754
Test 2, Discrete, 128 points: Avg error in 100 points = 1.8116690247663865
Test 3, Discrete, 256 points: Avg error in 100 points = 0.9567222806708231
Test 4, Discrete, 512 points: Avg error in 100 points = 0.47621506473854025
Test 5, Discrete, 1024 points: Avg error in 100 points = 0.29908917788505035


# **Discussion**

This assignment was quite a bit more difficult than the others, luckily only one problem had to be solved! While I would have wanted to solve a bonus problem too, I cannot quite picture how I would have done it. To begin, one would have needed a suitable triangular mesh structure with vertices and edges. Each vertex would have as many basis functions as sides, and this would have to be considered with q. It would also have to be done in a higher dimension than 1 which feels like a hassle to implement, so I will be content with just this one solution this time.